# 인공지능 작사가 만들기

In [1]:
import glob
import re                  
import numpy as np         
import tensorflow as tf  
import os
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel//lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[:Nicki Minaj]', 'Young money [Verse 1: Jason Derulo] (?) thousand different favors', 'I wish that I could (?)']


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

[:Nicki Minaj]
Young money [Verse 1: Jason Derulo] (?) thousand different favors
I wish that I could (?)
No i ain't got no dinner plans
So you should bring all your friends
I swear to (?) that you my type Noooo [Hook :Jason Derulo]
Shimmy shimmy she, shimmy she, shimmy yah
Swalla-la-la, swalla-la-la, swalla-la-la
Shimmy shimmy she, shimmy she, shimmy yah
Swalla-la-la, swalla-la-la, swalla-la-la


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> nicki minaj <end>',
 '<start> young money verse jason derulo ? thousand different favors <end>',
 '<start> i wish that i could ? <end>',
 '<start> no i ain t got no dinner plans <end>',
 '<start> so you should bring all your friends <end>',
 '<start> i swear to ? that you my type noooo hook jason derulo <end>',
 '<start> shimmy shimmy she , shimmy she , shimmy yah <end>',
 '<start> swalla la la , swalla la la , swalla la la <end>',
 '<start> shimmy shimmy she , shimmy she , shimmy yah <end>',
 '<start> swalla la la , swalla la la , swalla la la <end>']

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre', maxlen = 15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   0    0    0 ... 1202 2639    3]
 [   0    0    0 ...  635 3501    3]
 [   0    0    0 ...  104   43    3]
 ...
 [   5   22  713 ... 4178    4    3]
 [   5   22  713 ... 4178    4    3]
 [   5   22  713 ... 4178   20    3]] <keras_preprocessing.text.Tokenizer object at 0x7fb00d1d7dd0>


In [6]:
print(tensor[:3, :10])

[[   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    2  278  144  520 3390 6042]
 [   0    0    0    0    0    0    0    2    5  259]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   0    0    0    0    0    0    0    0    0    0    0    2 1202 2639]
[   0    0    0    0    0    0    0    0    0    0    2 1202 2639    3]


In [9]:
enc_train, enc_test, dec_train, dec_test = train_test_split(src_input, tgt_input, test_size = 0.2)

In [10]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [11]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [18]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size_1, hidden_size_2):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size_1, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size_2, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size_1 = 1024
hidden_size_2 = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size_1, hidden_size_2)

In [19]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-2.82842549e-04, -2.64262722e-04,  1.11803049e-04, ...,
         -2.01830233e-04, -1.64114768e-04, -2.10220343e-04],
        [-6.39711448e-04, -6.53163763e-04,  2.88058829e-04, ...,
         -4.49752202e-04, -4.11199435e-04, -5.05181844e-04],
        [-9.69419489e-04, -1.08004035e-03,  4.88436781e-04, ...,
         -6.71039452e-04, -6.77273725e-04, -8.11051170e-04],
        ...,
        [-1.96915629e-04, -2.08197488e-03,  2.24011983e-05, ...,
         -4.81997413e-04,  9.84153477e-04,  3.36059952e-06],
        [ 9.22939580e-05, -1.46578741e-03, -2.69355442e-05, ...,
         -1.94685141e-04,  1.68212957e-03,  2.21399925e-04],
        [ 2.07569203e-04, -1.00681616e-03, -3.11699077e-05, ...,
         -7.82459174e-05,  1.82667549e-03,  6.90511733e-05]],

       [[-2.82842549e-04, -2.64262722e-04,  1.11803049e-04, ...,
         -2.01830233e-04, -1.64114768e-04, -2.10220343e-04],
        [-6.39711448e-04, -6.53163763e-04,  2

In [20]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  6144512   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  6295552   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  25174016  
_________________________________________________________________
dense_1 (Dense)              multiple                  24590049  
Total params: 62,204,129
Trainable params: 62,204,129
Non-trainable params: 0
_________________________________________________________________


In [21]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 168s 306ms/step - loss: 3.6556
Epoch 2/10
549/549 [==============================] - 165s 301ms/step - loss: 3.1336
Epoch 3/10
549/549 [==============================] - 160s 291ms/step - loss: 2.8953
Epoch 4/10
549/549 [==============================] - 163s 298ms/step - loss: 2.6683
Epoch 5/10
549/549 [==============================] - 168s 306ms/step - loss: 2.5220
Epoch 6/10
549/549 [==============================] - 156s 284ms/step - loss: 2.2909
Epoch 7/10
549/549 [==============================] - 160s 292ms/step - loss: 2.1103
Epoch 8/10
549/549 [==============================] - 155s 282ms/step - loss: 1.9391
Epoch 9/10
549/549 [==============================] - 162s 295ms/step - loss: 1.7789
Epoch 10/10
549/549 [==============================] - 166s 303ms/step - loss: 1.6303


In [22]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [23]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i need you , i need you , oh <end> '

저번에 배웠던 RNN의 연장선상에 있는 프로젝트였다. 이번에 느낀건 자연어를 tensor로 바꿔줘서 계산한다는 점이 더 많이 느끼게 되었다.     
그리고 생각보다 model을 실행하는 시간이 꽤 걸렸다. 또한 가사 텍스트 전부를 넣고 돌리게 되면 OOM이 일어났었다. NLP라는 분야가 생각보다 GPU성능이 중요하다는 것을 다시금 깨달았다.